# Машинное обучение, РЭШ

## Практическое задание 2. Exploratory Data Analysis и линейная регрессия

### Общая информация
Дата выдачи: 05.11.2022

Дедлайн: 17:00MSK 17.11.2022

### О задании
В этом задании мы попытаемся научиться анализировать данные и выделять из них полезные признаки. Мы также научимся пользоваться `seaborn` и `sklearn`, а заодно привыкнем к основным понятиям машинного обучения.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов. Проверяющий имеет право снизить оценку за неэффективную реализацию или неопрятные графики.

**Обратите внимание**, что в каждом разделе домашнего задания есть оцениваниемые задачи и есть вопросы. Вопросы дополняют задачи и направлены на то, чтобы проинтерпретировать или обосновать происходящее. Код без интерпретации не имеет смысла, поэтому отвечать на вопросы обязательно — за отсутствие ответов мы будем снижать баллы за задачи. Если вы ответите на вопросы, но не напишете корректный код к соответствующим оцениваемым задачам, то баллы за такое выставлены не будут.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

### Формат сдачи
Задания сдаются на my.nes в виде Jupyter-notebook файла.

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

Оценка: xx.

В этом ноутбуке используется библиотека `folium` для визуализации карт. Она работает в google colab!

In [ ]:
import folium

# m = folium.Map(location=(55.7522200, 37.6155600), zoom_start=10)
# m

Если вы всё сделали правильно, то выше должна открыться карта Москвы.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error
# import matplotlib.pyplot as plt
import seaborn as sns

# %matplotlib inline
sns.set_theme(style="darkgrid")

def pprint(*args, **kwargs):
    print(">>", *args, **kwargs)

## Часть 0. Подготовка (1 балл)

**Задание 1 (1 балл)**. Мы будем работать с данными из соревнования [New York City Taxi Trip Duration](https://www.kaggle.com/c/nyc-taxi-trip-duration/overview), в котором нужно было предсказать длительность поездки на такси. Скачайте обучающую выборку из этого соревнования и загрузите ее:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df = pd.read_csv("data/nyc-taxi-trip-duration/train.csv")
df.info()

In [ ]:
# check on nans
df.isna().any()

Обратите внимание на колонки `pickup_datetime` и `dropoff_datetime`. `dropoff_datetime` был добавлена организаторами только в обучающую выборку, то есть использовать эту колонку нельзя, давайте удалим ее. В `pickup_datetime` записаны дата и время начала поездки. Чтобы с ней было удобно работать, давайте преобразуем даты в `datetime`-объекты

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df = df.drop(columns="dropoff_datetime")
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], format="%Y-%m-%d %H:%M:%S")

В колонке `trip_duration` записано целевое значение, которое мы хотим предсказывать. Давайте посмотрим на распределение таргета в обучающей выборке. Для этого нарисуйте его гистограмму:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
import plotly.express as px

fig = px.histogram(data_frame=df, x="trip_duration", log_y=True)
fig.update_layout(title_text="Distribution of trip duration (log scale)", xaxis_title = "Trip duration", yaxis_title = "Count")
fig.show()

In [ ]:
pprint("an extremely heave tail for the target, a few obvious large outliers spotted!")

**Вопрос**: Что можно сказать о целевой переменной по гистограмме её значений?

В соревновании в качестве метрики качества использовалось RMSLE:
$$\text{RMSLE}(X, y, a) = \sqrt{\frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2}$$

**Вопрос**: Как вы думаете, почему авторы соревнования выбрали именно RMSLE, а не RMSE?

In [ ]:
pprint("as the rmsle metric is more robust with outliers, while rmse is too sensitive")

На семинаре мы рассматривали несколько моделей линейной регрессии в `sklearn`, но каждая из них оптимизировала среднеквадратичную ошибку (MSE), а не RMSLE. Давайте проделаем следующий трюк: будем предсказывать не целевую переменную, а ее *логарифм*. Обозначим $\hat{y}_i = \log{(y_i + 1)}$ — модифицированный таргет, а $\hat{a}(x_i)$ — предсказание модели, которая обучалась на $\hat{y}_i$, то есть логарифм таргета. Чтобы предсказать исходное значение, мы можем просто взять экспоненту от нашего предсказания: $a(x_i) = \exp(\hat{a}(x_i)) - 1$.

**Вопрос**: Покажите, что оптимизация RMSLE для модели $a$ эквивалентна оптимизации MSE для модели $\hat{a}$.

**Доказательство**: ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

$$\text{RMSLE}(X, y, a) = \sqrt{\frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2}$$
$$\hat{y}_i = \log{(y_i + 1)}$$
$$\hat{a}_i = \log{(a(x_i) + 1)} $$
$$\Rightarrow \text{RMSLE}(X, y, a) = \sqrt{\frac{1}{\ell}\sum_{i=1}^{\ell} \big(\hat{y}_i - \hat{a}_i\big)^2} = \text{RMSE}(X, \hat{y}, \hat{a}) $$


Итак, мы смогли свести задачу оптимизации RMSLE к задаче оптимизации MSE, которую мы умеем решать! Кроме того, у логарифмирования таргета есть еще одно полезное свойство. Чтобы его увидеть, добавьте к нашей выборке колонку `log_trip_duration` (воспользуйтесь `np.log1p`) и нарисуйте гистограмму модифицированного таргета по обучающей выборке. Удалите колонку со старым таргетом.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["log_trip_duration"] = np.log1p(df["trip_duration"])
df = df.drop(columns="trip_duration")

fig = px.histogram(data_frame=df, x="log_trip_duration")
fig.update_layout(title_text="Distribution of log trip duration", xaxis_title = "Log of trip duration", yaxis_title = "Count", bargap=0.1)
fig.show()

Чтобы иметь некоторую точку отсчета, давайте посчитаем значение метрики при наилучшем константном предсказании:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# the best constant forecast is the mean
best_const_forecast = np.full((len(df["log_trip_duration"]), 1), df["log_trip_duration"].mean())
pprint(f"rmse with the best constant forecast: {root_mean_squared_error(df[['log_trip_duration']], best_const_forecast):.3f}")

## Часть 1. Изучаем `pickup_datetime` (2 балла)

**Задание 2 (0.25 баллов)**. Для начала давайте посмотрим, сколько всего было поездок в каждый из дней. Постройте график зависимости количества поездок от дня в году (например, можно воспользоваться `sns.countplot`):

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
fig = px.histogram(data_frame=df, x=df["pickup_datetime"].dt.date)
fig.update_layout(title_text="Distribution of trips during the year", xaxis_title = "Date", yaxis_title = "Count", bargap=0.1)
fig.show()

**Вопрос**: Вы, вероятно, заметили, что на графике есть 2 периода с аномально маленькими количествами поездок. Вычислите, в какие даты происходили эти скачки вниз и найдите информацию о том, что происходило в эти дни в Нью-Йорке.<br>
**Answer**:
* On January 23, 2016, New York saw a massive blizzard
* On May 28, 2016, a tropical storm Bonnie brought heavy rain up to the northeast coast

Нарисуйте графики зависимости количества поездок от дня недели и от часов в сутках (воспользуйтесь `sns.relplot`):

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
counts = df.groupby(
    [df["pickup_datetime"].dt.day_name().rename("weekday"), df["pickup_datetime"].dt.hour.rename("hour")]
).size().reset_index(name="count")

fig = px.scatter(data_frame=counts, x="weekday", y="count", color="hour", color_continuous_scale="Viridis",
                 category_orders={"weekday": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}
                 )
fig.update_layout(title_text="Distribution of trips over weekday and hour", xaxis_title = "Weekday", yaxis_title = "Count")
fig.show()
del counts

**Задание 3 (0.5 баллов)**. Нарисуйте на одном графике зависимости количества поездок от часа в сутках для разных месяцев (разные кривые, соответствующие разным месяцам, окрашивайте в разные цвета, воспользуйтесь `hue` в `sns.relplot`). Аналогично нарисуйте зависимости количества поездок от часа в сутках для разных дней недели.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# counts on hour and month
counts = df.groupby(
    [df["pickup_datetime"].dt.month_name().rename("month"), df["pickup_datetime"].dt.hour.rename("hour")]
).size().reset_index(name="count")
fig1 = px.scatter(data_frame=counts, x="hour", y="count", color="month")
fig1.update_layout(title_text="Distribution of trips over hour and month", xaxis_title = "Hour", yaxis_title = "Count", legend_title="Month")
fig1.show()
del counts

In [ ]:
# counts on hour and day of the week
counts = df.groupby(
    [df["pickup_datetime"].dt.day_name().rename("weekday"), df["pickup_datetime"].dt.hour.rename("hour")]
).size().reset_index(name="count")
fig2 = px.scatter(data_frame=counts, x="hour", y="count", color="weekday")
fig2.update_layout(title_text="Distribution of trips over hour and weekday", xaxis_title = "Hour", yaxis_title = "Count", legend_title="Weekday")
fig2.show()
del counts

**Вопрос**: Какие выводы можно сделать, основываясь на графиках выше? Выделяются ли какие-нибудь дни недели? Месяца? Время суток? С чем это связано?<br>
**Answer**:
1) Across all months and days of the week in the sample, the total number of trips follows a daily cycle: demand is lowest late at night, increases around 7–8 a.m. during the morning rush hour, peaks in the evening, and then gradually declines.
2) Nighttime trips are dominated by weekends, while morning trips are more typical on weekdays.
3) January shows significantly fewer trips compared to other months in the sample, possibly a post-holiday effect.
4) On weekends, morning trip volumes are substantially lower than on other days of the week.

**Задание 4 (0.5 баллов)**. Разбейте выборку на обучающую и тестовую в отношении 7:3. По обучающей выборке нарисуйте график зависимости среднего логарифма времени поездки от дня недели. Затем сделайте то же самое, но для часа в сутках и дня в году.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3, random_state=1998)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def add_to_subplot(to_add: go.Figure = None, subplot: go.Figure = None, row: int = None, col: int = None,
                   color: str = None, opacity: float = None) -> None:
    for trace in to_add.data:
        if color is not None:
            trace.marker.color = color
        if opacity is not None:
            trace.opacity = opacity
        subplot.add_trace(trace, row=row, col=col)

fig = make_subplots(rows=1, cols=3, subplot_titles=["Mean log trip duration on weekday", "Mean log trip duration on hour", "Mean log trip duration on date"])

# counts on weekday
counts = df.groupby(df["pickup_datetime"].dt.day_name().rename("weekday")).agg(mean_log_trip_duration=("log_trip_duration", "mean")).reset_index(drop=False)
fig1 = px.scatter(data_frame=counts, x="weekday", y="mean_log_trip_duration", category_orders={"weekday": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
add_to_subplot(fig1, fig, row=1, col=1)
del counts, fig1

# counts on hour
counts = df.groupby(df["pickup_datetime"].dt.hour.rename("hour")).agg(mean_log_trip_duration=("log_trip_duration", "mean")).reset_index(drop=False)
fig2 = px.line(data_frame=counts, x="hour", y="mean_log_trip_duration")
add_to_subplot(fig2, fig, row=1, col=2)
del counts, fig2

# counts on date
counts = df.groupby(df["pickup_datetime"].dt.dayofyear.rename("date")).agg(mean_log_trip_duration=("log_trip_duration", "mean")).reset_index(drop=False)
fig3 = px.line(data_frame=counts, x="date", y="mean_log_trip_duration")
add_to_subplot(fig3, fig, row=1, col=3)
del counts, fig3


fig.update_yaxes(title_text="Mean of log trip duration")
fig.update_xaxes(title_text="Weekday", categoryorder="array", categoryarray=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], row=1, col=1)
fig.update_xaxes(title_text="Hour", row=1, col=2)
fig.show()

**Вопрос**: Похожи ли графики зависимости таргета от дня недели и от часа в сутках на аналогичные графики для количества поездок? Почему? Что происходит со средним таргетом в те два аномальных периода, что мы видели выше? Почему так происходит? Наблюдаете ли вы какой-нибудь тренд на графике зависимости `log_trip_duration` от номера дня в году?<br>
**Answer**:
1) The target's dependence on the day of the week is similar in both cases: the curves are hump-shaped, with a peak on Thursday. The same pattern holds for the hourly dependence: after a nighttime decline until around 5 a.m., there is a sharp increase during the morning rush hour, stabilizing by 3 p.m. The similarity is explained by the fact that both targets reflect the same underlying phenomenon, measured in different units, either the number of trips or the logarithm of total trip distance. An increase in one naturally corresponds to an increase in the other.
2) We see spikes around the date of blizzard (either low demand on trips, or long duration because of snow), and storm.
3) Yes, an upward trend is observed.

Добавьте следующие признаки на основе `pickup_datetime`:
1. День недели
2. Месяц
3. Час
4. Является ли период аномальным (два бинарных признака, соответствующие двум аномальным периодам)
5. Номер дня в году

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["weekday"] = df["pickup_datetime"].dt.day_name()
df["month"] = df["pickup_datetime"].dt.month
df["hour"] = df["pickup_datetime"].dt.hour
df["day_year_number"] = df["pickup_datetime"].dt.dayofyear

# masks on anomalies
df["if_anomaly1"]  = df["pickup_datetime"].dt.date.astype("str").isin(["2016-01-23", "2016-01-24", "2016-01-25", "2016-01-26"])
df["if_anomaly2"]  = df["pickup_datetime"].dt.date.astype("str").isin(["2016-05-28", "2016-05-29", "2016-05-30"])

In [ ]:
# add created variables to subsamples
train_df, test_df = train_test_split(df, test_size=0.3, random_state=1998)

# fix categorical and numerical features
categorical_f1 = ["if_anomaly1", "if_anomaly2", "weekday"]
numerical_f1 = ["month", "hour", "day_year_number"]

x_train1, y_train1 = train_df[categorical_f1 + numerical_f1], train_df["log_trip_duration"]
x_test1, y_test1 = test_df[categorical_f1 + numerical_f1], test_df["log_trip_duration"]

Итак, мы уже создали некоторое количество признаков.

**Вопрос**: Какие из признаков стоит рассматривать как категориальные, а какие - как численные? Почему?<br>
**Answer**: Among categorical features are a dummy on anomaly day and name of the day. Numerical features are all the rest: month number, hour, day number in year.


**Задание 5 (0.75 баллов)**. Обучите `Ridge`-регрессию с параметрами по умолчанию, закодировав все категориальные признаки с помощью `OneHotEncoder`. Численные признаки отмасштабируйте с помощью `StandardScaler`. Используйте только признаки, которые мы выделили в этой части задания.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV


class MyLinearRegressionClass():

    def __init__(
            self,
            x_train: pd.DataFrame,
            y_train: pd.Series,
            x_test: pd.DataFrame,
            y_test: pd.Series,
            categorical_f: list,
            numerical_f: list,
            estimator=Ridge()
    ):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.categorical_f = categorical_f
        self.numerical_f = numerical_f
        self.estimator = estimator
        self.model = None

    def gs_linear_regression(self):
        # ohe of categorical (drop the first to deal with linear dependence) and scaling of numerical
        column_transformer = ColumnTransformer([
            ("ohe", OneHotEncoder(drop="first", handle_unknown="ignore"), self.categorical_f),
            ("scaler", StandardScaler(), self.numerical_f)
        ])

        # pack all steps into pipe
        pipe = Pipeline([
            ("column_transformer", column_transformer),
            ("estimator", self.estimator)
        ])

        param_grid = {"estimator__alpha": list(np.logspace(-5, 5, num = 11))}

        grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=-1, cv=5, scoring="neg_root_mean_squared_error")
        grid_search.fit(self.x_train, self.y_train)

        self.model = grid_search.best_estimator_

        pprint(f"best params: {grid_search.best_params_}")
        pprint(f"best cv rmse: {-grid_search.best_score_:.4f}")

        train_pred = self.model.predict(self.x_train)
        baseline_train_pred = np.full(len(self.y_train), self.y_train.mean())

        test_pred = self.model.predict(self.x_test)
        baseline_test_pred = np.full(len(self.y_test), self.y_train.mean())

        pprint(f"obtained rmse on train {root_mean_squared_error(self.y_train, train_pred):.4f} (vs on baseline predict {root_mean_squared_error(self.y_train, baseline_train_pred):.4f})")
        pprint(f"obtained rmse on test {root_mean_squared_error(self.y_test, test_pred):.4f} (vs on baseline predict {root_mean_squared_error(self.y_test, baseline_test_pred):.4f})")


    def linear_regression_predict(self):
        test_pred = self.model.predict(self.x_test)
        pprint(f"obtained rmse on test {root_mean_squared_error(self.y_test, test_pred):.4f}")


    def linear_regression(self, alpha: float = None) -> pd.Series:
        # ohe of categorical (drop the first to deal with linear dependence) and scaling of numerical
        column_transformer = ColumnTransformer([
            ("ohe", OneHotEncoder(drop="first", handle_unknown="ignore"), self.categorical_f),
            ("scaler", StandardScaler(), self.numerical_f)
        ])

        # if alpha is given, set it in estimator (without mutating the passed object)
        estimator = self.estimator
        if alpha is not None:
            estimator = clone(estimator).set_params(alpha=alpha)

        # pack all steps into pipe
        pipe = Pipeline([
            ("column_transformer", column_transformer),
            ("estimator", estimator)
        ])

        self.model = pipe.fit(self.x_train, self.y_train)

        train_pred = self.model.predict(self.x_train)
        baseline_train_pred = np.full(len(self.y_train), self.y_train.mean())

        test_pred = self.model.predict(self.x_test)
        baseline_test_pred = np.full(len(self.y_test), self.y_train.mean())

        pprint(f"obtained rmse on train {root_mean_squared_error(self.y_train, train_pred):.4f} (vs on baseline predict {root_mean_squared_error(self.y_train, baseline_train_pred):.4f})")
        pprint(f"obtained rmse on test {root_mean_squared_error(self.y_test, test_pred):.4f} (vs on baseline predict {root_mean_squared_error(self.y_test, baseline_test_pred):.4f})")

         # return residuals on train
        return self.y_train - train_pred

In [ ]:
lr1 = MyLinearRegressionClass(x_train=x_train1, y_train=y_train1, x_test=x_test1, y_test=y_test1, categorical_f=categorical_f1, numerical_f=numerical_f1, estimator=Ridge())
lr1.linear_regression()

;## Часть 2. Изучаем координаты (3 балла)
Мы уже очень хорошо изучили данные о времени начала поездки, давайте теперь посмотрим на информацию о координатах начала и конца поездки. Мы подготовили для вас функцию, которая на карте рисует точки начала или конца поездки. Примеры ее вызова вы найдете ниже. Обратите внимание, что в эту функцию мы передаем лишь небольшой кусочек данных, посколько иначе функция будет работать очень долго

In [ ]:
def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location)

    for _, row in data.iterrows():
        folium.Circle(
            radius=100,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m

In [ ]:
show_circles_on_map(df.sample(10000), "pickup_latitude", "pickup_longitude", "blue")

In [ ]:
show_circles_on_map(df.sample(10000), "dropoff_latitude", "dropoff_longitude", "blue")

**Вопрос**: Какие две точки выделяются на карте?<br>
**Answer**: two airports (JFK and EWR)

**Задание 6 (0.75 балл)**. Как мы все прекрасно помним, $t = s / v_{\text{ср}}$, поэтому очевидно, что самым сильным признаком будет расстояние, которое необходимо проехать. Мы не можем посчитать точное расстояние, которое необходимо преодолеть такси, но мы можем его оценить, посчитав кратчайшее расстояние между точками начала и конца поездки. Чтобы корректно посчитать расстояние между двумя точками на Земле, можно использовать функцию `haversine`. Также можно воспользоваться кодом с первого семинара. Посчитайте кратчайшее расстояние для объектов и запишите его в колонку `haversine`:

In [ ]:
from haversine import haversine

def distance(x):
    return haversine((x["pickup_latitude"], x["pickup_longitude"]), (x["dropoff_latitude"], x["dropoff_longitude"]))

df["haversine"] = df.apply(distance, axis = 1)
df["haversine"].describe()

Так как мы предсказываем логарифм времени поездки и хотим, чтобы наши признаки были линейно зависимы с этой целевой переменной, нам нужно логарифмировать расстояние: $\log t = \log s - \log{v_{\text{ср}}}$. Запишите логарифм `haversine` в отдельную колонку:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["log_haversine"] = np.log1p(df["haversine"])
df["log_haversine"].describe()

Убедимся, что логарифм расстояния лучше коррелирует с нашим таргетом, чем просто расстояние:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df[["haversine", "log_haversine", "log_trip_duration"]].corr()

**Задание 7 (0.75 балла)**. Давайте изучим среднюю скорость движения такси. Посчитайте среднюю скорость для каждого объекта обучающей выборки, разделив `haversine` на `trip_duration`, и нарисуйте гистограмму ее распределения

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["mean_speed"] = df["haversine"] / np.exp(df["log_trip_duration"])

# add created variables to subsamples
train_df, test_df = train_test_split(df, test_size=0.3, random_state=1998)

fig = px.histogram(train_df, x="mean_speed", log_y=True)
fig.update_layout(title="Distribution of mean speed", xaxis_title="Mean speed", yaxis_title="Count")
fig.show()

Как можно видеть по гистограмме, для некоторых объектов у нас получились очень больше значения скоростей. Нарисуйте гистограмму по объектам, для которых значение скорости получилось разумным (например, можно не включать рассмотрение объекты, где скорость больше некоторой квантили):

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
fig = px.histogram(train_df, x="mean_speed", range_x=[0, np.percentile(train_df["mean_speed"], 99)])
fig.update_layout(title="Distribution of mean speed", xaxis_title="Mean speed", yaxis_title="Count")
fig.show()

Для каждой пары (день недели, час суток) посчитайте медиану скоростей. Нарисуйте с помощью `sns.heatmap` график, где по осям будут дни недели и часы, а в качестве значения функции - медиана скорости

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
counts = df.groupby(["weekday", "hour"]).agg(median_speed=("mean_speed", "median")).reset_index(drop=False)
fig = px.imshow(
    pd.pivot(counts, index="weekday", columns="hour", values="median_speed").reindex(
        ["Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday"]
    ))
fig.update_layout(title="Distribution of median speed over weekday and hour", xaxis_title="Hour", yaxis_title="Weekday")
fig.show()

Не забудьте удалить колонку со значением скорости из данных!

**Вопрос**: Почему значение скорости нельзя использовать во время обучения?<br>
**Answer**: because for new data we won't have `log_trip_duration`.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df = df.drop(columns=["mean_speed"])

**Вопрос**: Посмотрите внимательно на график и скажите, в какие моменты времени скорость минимальна; максимальна.<br>
**Answer**: 
1) Speed is the lowest between 9 a.m. and 12 p.m. from Tuesday to Friday, and between 1 p.m. and 3 p.m. from Tuesday to Thursday.
2) Speed reaches its maximum on Sunday at 4 a.m. and on Monday at 6 a.m.

Создайте признаки "поездка совершается в период пробок" и "поездка совершается в период свободных дорог" (естественно, они не должен зависеть от скорости!):

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["high_traffic"] = ((df["weekday"].isin(["Tuesday", "Wednesday", "Thursday", "Friday"])) & (df["hour"].isin(range(9, 13))) |
                 (df["weekday"].isin(["Tuesday", "Wednesday", "Thursday"])) & (df["hour"].isin(range(13, 16))))
df["small_traffic"] = ((df["weekday"].isin(["Sunday"])) & (df["hour"] == 4)) | (df["weekday"].isin(["Monday"]) & (df["hour"] == 6))
df[["high_traffic", "small_traffic"]].value_counts()

**Задание 8 (0.25 балла)**. Как уже было замечено выше, на карте выделяются две точки вдали от Манхэттена. Для каждой из них добавьте в выборку два признака: началась ли поездка в ней и закончилась ли она в ней.

In [ ]:
# check coordinates of jfk and ewr
jfk_coordinates = [40.640864, -73.790531]
ewr_coordinates = [40.690345, -74.175100]

m = folium.Map(location=[sum(x) / 2 for x in zip(jfk_coordinates, ewr_coordinates)])
for coord, color in [(jfk_coordinates, "blue"), (ewr_coordinates, "red")]:
    folium.CircleMarker(location=coord, radius=15, color=color, fill=True).add_to(m)
# m

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
def if_close_airport(coordinates: tuple = None, coordinates_airport: tuple = None, radius: float = None):
    return haversine(coordinates, coordinates_airport) < radius

radius_km = 2.
df["jfk_dropoff"] = df.apply(lambda row: if_close_airport(coordinates=(row["dropoff_latitude"], row["dropoff_longitude"]),
                                                          coordinates_airport=tuple(jfk_coordinates), radius=radius_km), axis=1)
df["jfk_pickup"] = df.apply(lambda row: if_close_airport(coordinates=(row["pickup_latitude"], row["pickup_longitude"]),
                                                          coordinates_airport=tuple(jfk_coordinates), radius=radius_km), axis=1)
df["ewr_dropoff"] = df.apply(lambda row: if_close_airport(coordinates=(row["dropoff_latitude"], row["dropoff_longitude"]),
                                                          coordinates_airport=tuple(ewr_coordinates), radius=radius_km), axis=1)
df["ewr_pickup"] = df.apply(lambda row: if_close_airport(coordinates=(row["pickup_latitude"], row["pickup_longitude"]),
                                                          coordinates_airport=tuple(ewr_coordinates), radius=radius_km), axis=1)


Для каждого из созданных признаков нарисуйте "ящик с усами" (`sns.boxplot`) распределения логарифма времени поездки

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
fig = make_subplots(rows=1, cols=2, subplot_titles=["Boxplot of log trip duration when high traffic", "Boxplot of log trip duration when small traffic"])

fig1 = px.box(data_frame=df, x="high_traffic", y="log_trip_duration")
add_to_subplot(fig1, fig, row=1, col=1)

fig2 = px.box(data_frame=df, x="small_traffic", y="log_trip_duration")
add_to_subplot(fig2, fig, row=1, col=2)

fig.update_yaxes(title_text="Log trip duration")
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Boxplot of log trip duration when JFK pickup", "Boxplot of log trip duration when JFK dropoff"])

fig1 = px.box(data_frame=df, x="jfk_pickup", y="log_trip_duration")
add_to_subplot(fig1, fig, row=1, col=1)

fig2 = px.box(data_frame=df, x="jfk_dropoff", y="log_trip_duration")
add_to_subplot(fig2, fig, row=1, col=2)

fig.update_yaxes(title_text="Log trip duration")
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Boxplot of log trip duration when EWR pickup", "Boxplot of log trip duration when EWR dropoff"])

fig3 = px.box(data_frame=df, x="ewr_pickup", y="log_trip_duration")
add_to_subplot(fig3, fig, row=1, col=1)

fig4 = px.box(data_frame=df, x="ewr_dropoff", y="log_trip_duration")
add_to_subplot(fig4, fig, row=1, col=2)

fig.update_yaxes(title_text="Log trip duration")
fig.show()

**Вопрос**: судя по графикам, как вы думаете, хорошими ли получились эти признаки?<br>
**Answer**: yes, even though they look close

<img src="https://www.dropbox.com/s/xson9nukz5hba7c/map.png?raw=1" align="right" width="20%" style="margin-left: 20px; margin-bottom: 20px">

**Задание 9 (1 балл)**. Сейчас мы почти что не используем сами значения координат. На это есть несколько причин: по отдельности рассматривать широту и долготу не имеет особого смысла, стоит рассматривать их вместе. Во-вторых, понятно, что зависимость между нашим таргетом и координатами не линейная. Чтобы как-то использовать координаты, можно прибегнуть к следующему трюку: обрамим область с наибольшим количеством поездок прямоугольником (как на рисунке). Разобьем этот прямоугольник на ячейки. Каждой точке сопоставим номер ее ячейки, а тем точкам, что не попали ни в одну из ячеек, сопоставим значение -1.

Напишите трансформер, который сначала разбивает показанную на рисунке область на ячейки, а затем создает два признака: номер ячейки, в которой началась поездка, и номер ячейки, в которой закончилась поездка. Количество строк и столбцов выберите самостоятельно.

Обратите внимание, что все вычисления должны быть векторизованными, трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики (если они вдруг нужны) нужно считать только по обучающей выборке в методе `fit`:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MapGridTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            n_cols: int,
            n_rows: int,
            up_right_corner: tuple,
            down_left_corner: tuple
    ):
        self.n_cols = n_cols
        self.n_rows = n_rows
        self.up_right_corner = up_right_corner
        self.down_left_corner = down_left_corner

    def cell_definer(self, longitudes: pd.Series, latitudes: pd.Series):
        # coordinates of borders
        col_borders = np.linspace(self.down_left_corner[0], self.up_right_corner[0], self.n_cols + 1)
        row_borders = np.linspace(self.down_left_corner[1], self.up_right_corner[1], self.n_rows + 1)

        # get number of cell
        index_x = np.searchsorted(col_borders, longitudes, "right")
        index_y = np.searchsorted(row_borders, latitudes, "right")

        # state of being inside the rectangle
        inside_pos = (index_y < self.n_rows + 1) & (index_y > 0) & (index_x < self.n_cols + 1) & (index_x > 0)
        # by default, all are out of rectangle, hence have -1
        cells = np.full(longitudes.shape, -1)
        cells[inside_pos] = (index_y[inside_pos] - 1) * self.n_cols + (index_x[inside_pos] - 1)

        if np.unique(cells).shape[0] - 1 != self.n_cols * self.n_rows:
           pprint("dimension error!")
        return cells

    def transformer(self, df: pd.DataFrame = None):
        df["cell_pickup"] = self.cell_definer(df["pickup_longitude"], df["pickup_latitude"])
        df["cell_dropoff"] = self.cell_definer(df["dropoff_longitude"], df["dropoff_latitude"])

        return df

In [ ]:
class_mgd = MapGridTransformer(n_cols=5, n_rows=8, up_right_corner=(-73.931280, 40.794559), down_left_corner=(-74.023999, 40.701835))
df = class_mgd.transformer(df=df)
df[["cell_pickup", "cell_dropoff"]].value_counts()

# add created variables to subsamples
train_df, test_df = train_test_split(df, test_size=0.3, random_state=1998)

**Задание 10 (0.25 балла)**. Обучите `Ridge`-регрессию со стандартными параметрами на признаках, которые мы выделили к текущему моменту. Категориальные признаки закодируйте через one-hot-кодирование, числовые признаки отмасштабируйте.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# fix categorical and numerical features
categorical_f2 = ["if_anomaly1", "if_anomaly2", "weekday", "high_traffic", "small_traffic", "jfk_pickup", "jfk_dropoff", "ewr_pickup", "ewr_dropoff"]
numerical_f2 = ["month", "hour", "day_year_number", "log_haversine", "cell_pickup", "cell_dropoff"]

x_train2, y_train2 = train_df[categorical_f2 + numerical_f2], train_df["log_trip_duration"]
x_test2, y_test2 = test_df[categorical_f2 + numerical_f2], test_df["log_trip_duration"]
lr2 = MyLinearRegressionClass(x_train=x_train2, y_train=y_train2, x_test=x_test2, y_test=y_test2, categorical_f=categorical_f2, numerical_f=numerical_f2)
lr2.linear_regression();

In [ ]:
lr1.linear_regression();

## Часть 3. Изучаем оставшиеся признаки (1 балл)

**Задание 11 (0.75 баллов)**. У нас осталось еще 3 признака, которые мы не исследовали: `vendor_id`, `passenger_count` и `store_and_fwd_flag`.

**Вопрос**: Подумайте, почему каждый из этих признаков может быть потенциально полезным.<br>
**Answer**:
1) `vendor_id`– an indicator identifying the carrier that operated the trip. It provides information on the relative popularity of carriers and their overall market share.
2) `passenger_count` – the number of passengers in the vehicle. It reflects overall customer volume per trip.
3) `store_and_fwd_flag` – a flag indicating whether the trip record was temporarily stored in the vehicle's memory before being transmitted to the carrier due to a lack of server connection (Y = stored and forwarded; N = transmitted immediately). It captures potential delays in data transmission.

Посчитайте, сколько есть уникальных значений у каждого из этих признаков:

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df[["vendor_id", "passenger_count", "store_and_fwd_flag"]].nunique()

Постройте "ящики с усами" распределений логарифма времени поездки в зависимости от значений каждого из признаков

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
fig = make_subplots(rows=1, cols=3, subplot_titles=["Boxplot of log trip duration on vendor_id", "Boxplot of log trip duration on passenger count", "Boxplot of log trip duration on store flag"])

fig1 = px.box(data_frame=df, x="vendor_id", y="log_trip_duration")
add_to_subplot(fig1, fig, row=1, col=1)

fig2 = px.box(data_frame=df, x="passenger_count", y="log_trip_duration")
add_to_subplot(fig2, fig, row=1, col=2)

fig3 = px.box(data_frame=df, x="store_and_fwd_flag", y="log_trip_duration")
add_to_subplot(fig3, fig, row=1, col=3)

fig.update_yaxes(title_text="Log trip duration")
fig.show()

Переведите признаки `vendor_id` и `store_and_fwd_flag` в значения $\{0;1\}$

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df["vendor_id"] = df["vendor_id"].map({1: 0, 2: 1})
df["store_and_fwd_flag"] = df["store_and_fwd_flag"].map({"N": 0, "Y": 1})

**Вопрос**: Основываясь на графиках выше, как вы думаете, будут ли эти признаки сильными? <br>
**Answer**: Nope, these features are rather poor. As the distributions for `vendor_id` are nearly identical across carriers. Similarly, the distributions for `store_and_fwd_flag` show only minor differences. In contrast, for `passenger_count`, the distributions differ noticeably only in the cases of maximum occupancy and zero passengers.

**Задание 12 (0.25 баллов)**. Проверьте свои предположения, обучив модель в том числе и на этих трех признаках. Обучайте `Ridge`-регрессию со стандартными параметрами. Категориальные признаки закодируйте one-hot-кодированием, а численные отмасштабируйте.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# add created variables to subsamples
train_df, test_df = train_test_split(df, test_size=0.3, random_state=1998)

# fix categorical and numerical features
categorical_f3 = ["if_anomaly1", "if_anomaly2", "weekday", "high_traffic", "small_traffic", "jfk_pickup", "jfk_dropoff", "ewr_pickup", "ewr_dropoff", "vendor_id", "store_and_fwd_flag", "passenger_count"]
numerical_f3 = [
    "month", "hour", "day_year_number", "log_haversine", "cell_pickup", "cell_dropoff"]

x_train3, y_train3 = train_df[categorical_f3 + numerical_f3], train_df["log_trip_duration"]
x_test3, y_test3 = test_df[categorical_f3 + numerical_f3], test_df["log_trip_duration"]
lr3 = MyLinearRegressionClass(x_train=x_train3, y_train=y_train3, x_test=x_test3, y_test=y_test3, categorical_f=categorical_f3, numerical_f=numerical_f3)
res3 = lr3.linear_regression()

In [ ]:
lr2.linear_regression();

Если признаки не дали какого-то ощутимого улучшения метрики, их можно выбросить из данных.

## Часть 4. Улучшаем модель (3 балла)

**Задание 13 (1 балл)**. В наших данных есть нетипичные объекты: с аномально маленьким времени поездки, с очень большим пройденным расстоянием или очень большими остатками регрессии. В этом задании предлагается исключить такие объекты из обучающей выборки. Для этого нарисуйте гистограммы распределения упомянутых выше величин, выберите объекты, которые можно назвать выбросами, и очистите обучающую выборку от них.

Отметим, что хотя эти объекты и выглядят как выбросы, в тестовой выборке тоже скорее всего будут объекты с такими же странными значениями целевой переменной и/или признаков. Поэтому, возможно, чистка обучающей выборки приведёт к ухудшению качества на тесте. Тем не менее, всё равно лучше удалять выбросы из обучения, чтобы модель получалась более разумной и интерпретируемой.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
fig = make_subplots(rows=1, cols=3)

add_to_subplot(px.histogram(res3), fig, row=1, col=1, color="red")
train_df = train_df.drop(index=res3[(res3 > 2.) | (res3 < -2.)].index)
add_to_subplot(px.histogram(res3[(res3 < 2.) & (res3 > -2.)]), fig, row=1, col=1, color="blue")

add_to_subplot(px.histogram(data_frame=train_df, x="log_trip_duration"), fig, row=1, col=2, color="red")
train_df = train_df[(train_df["log_trip_duration"] > 4.) & (train_df["log_trip_duration"] < 9.)]
add_to_subplot(px.histogram(data_frame=train_df, x="log_trip_duration"), fig, row=1, col=2, color="blue")

add_to_subplot(px.histogram(data_frame=train_df, x="log_haversine"), fig, row=1, col=3, color="red")
train_df = train_df[train_df["log_haversine"] < 3.5]
add_to_subplot(px.histogram(data_frame=train_df, x="log_haversine"), fig, row=1, col=3, color="blue")

fig.update_layout(showlegend=False)
fig.show()

Сейчас у нас очень много категориальных признаков. В категориальных признаках могут содержаться редкие категории, обычно это плохо: модель сильно переобучается на таких примерах. Попробуйте объединить редкие категории в одну. Естественно, делать это нужно только для действительно редких категорий.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
for cat in categorical_f3:
    pprint(cat)
    for value, ratio in train_df[cat].value_counts(normalize=True).items():
        pprint(f"{value} {ratio:.6f}")
    print("")

In [ ]:
# let's put rare counts of passenger in one value
train_df["passenger_count"] = np.where(~train_df["passenger_count"].isin([1, 2]), "0or3+", train_df["passenger_count"])
test_df["passenger_count"] = np.where(~test_df["passenger_count"].isin([1, 2]), "0or3+", test_df["passenger_count"])

In [ ]:
train_df["passenger_count"].value_counts()

Обучите модель на очищенных данных и посчитайте качество на тестовой выборке.

In [ ]:
x_train4, y_train4 = train_df[categorical_f3 + numerical_f3], train_df["log_trip_duration"]
x_test4, y_test4 = test_df[categorical_f3 + numerical_f3], test_df["log_trip_duration"]

lr4 = MyLinearRegressionClass(x_train=x_train4, y_train=y_train4, x_test=x_test4, y_test=y_test4, categorical_f=categorical_f3, numerical_f=numerical_f3)
lr4.linear_regression();

In [ ]:
lr3.linear_regression();

**Задание 14 (1 балл)**. После OneHot-кодирования количество признаков в нашем датасете сильно возрастает. Посчитайте колиество признаков до и после кодирования категориальных признаков.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
pprint(f"before ohe: {len(categorical_f3)}")
pprint(f"after ohe: {OneHotEncoder().fit_transform(X=x_train3[categorical_f3]).shape[1]}")

Попробуйте обучить не `Ridge`-, а `Lasso`-регрессию. Какой метод лучше?

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
lr5 = MyLinearRegressionClass(x_train=x_train4, y_train=y_train4, x_test=x_test4, y_test=y_test4, categorical_f=categorical_f3, numerical_f=numerical_f3, estimator=Lasso())
lr5.linear_regression();

In [ ]:
lr4.linear_regression();

Разбейте обучающую выборку на обучающую и валидационную в отношении 8:2. По валидационной выборке подберите оптимальные значения параметра регуляризации (по логарифмической сетке) для `Ridge` и `Lasso`, на тестовой выборке измерьте качество лучшей полученной модели.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=1998)

x_train5, y_train5 = train_df[categorical_f3 + numerical_f3], train_df["log_trip_duration"]
x_test5, y_test5 = val_df[categorical_f3 + numerical_f3], val_df["log_trip_duration"]

In [ ]:
lr6 = MyLinearRegressionClass(x_train=x_train5, y_train=y_train5, x_test=x_test5, y_test=y_test5, categorical_f=categorical_f3, numerical_f=numerical_f3, estimator=Ridge())

# for alpha in list(np.logspace(-7, 5, num = 11)):
#     pprint(f"current alpha: {alpha}")
#     lr6.linear_regression(alpha=alpha);

# I want to try grid search instead
lr6.linear_regression();
lr6.gs_linear_regression();

# assess error on test
lr6.x_test = x_test4
lr6.y_test = y_test4
lr6.linear_regression_predict()

In [ ]:
lr7 = MyLinearRegressionClass(x_train=x_train5, y_train=y_train5, x_test=x_test5, y_test=y_test5, categorical_f=categorical_f3, numerical_f=numerical_f3, estimator=Lasso())

for alpha in list(np.logspace(-7, 5, num = 11)):
    pprint(f"current alpha: {alpha}")
    lr7.linear_regression(alpha=alpha);

Для каждого перебранного `alpha` для Lasso посчитайте количество нулевых весов в модели и нарисуйте график зависимости его от `alpha`. Как сильно придётся потерять в качестве, если мы хотим с помощью Lasso избавиться хотя бы от половины признаков?

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

<img src="https://www.dropbox.com/s/wp4jj0599np17lh/map_direction.png?raw=1" width="20%" align="right" style="margin-left: 20px">

**Задание 15 (1 балл)**. Часто бывает полезным использовать взаимодействия признаков (feature interactions), то есть строить новые признаки на основе уже существующих. Выше мы разбили карту Манхэттена на ячейки и придумали признаки "из какой ячейки началась поездка" и "в какой ячейке закончилась поездка".

Давайте попробуем сделать следующее: посчитаем, сколько раз встречается каждая возможная пара этих признаков в нашем датасете и выберем 100 самых частых пар. Закодируем поездки с этими частыми парами как категориальный признак, остальным объектам припишем -1. Получается, что мы закодировали, откуда и куда должно было ехать такси.

Также можете придумать ещё какой-нибудь способ сделать признаки про маршрут. Если эти признаки будут давать хороший прирост в качестве, то за это могут быть даны дополнительные бонусные баллы.

**Вопрос**: Почему такой признак потенциально полезный? Почему линейная модель не может самостоятельно "вытащить" эту информацию, ведь у нее в распоряжении есть признаки "из какой ячейки началась поездка" и "в какой ячейке закончилась поездка"?

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Заново обучите модель (`Ridge`, если она дала более высокое качество в предыдущих экспериментах, и `Lasso` иначе) на новых даннных и посчитайте качество на тестовой выборке

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ